In [1]:
import numpy as np
from sympy import *

# Define Type VII data
class VII():
    
    # Type VII data will have two attributes: direction and size
    def __init__(self, direction=np.array([0,0,1]), mould=1):
        self.direction = direction 
        self.mould = mould
        self.dimension = len(self.direction)
        A = self.direction
        if (A[0]==x and A[1]==y and A[2]==z and self.mould==0):
            self.type = 'nabla'
        elif (A[0]==0 and A[1]==0 and A[2]==0):
            self.type = 'intnsity'
        else:
            self.type = 'vector'
            
    x, y, z = symbols('x, y, z')
    
    # Define nabla function
    @staticmethod
    def nabla():
        return VII(np.array([x, y, z]), 0)
    
    # Convert 3D vector to VII format data
    @staticmethod
    def load_vector(vec):
        re1 = sum(i**2 for i in vec)**0.5
        if re1**2 != 0:
            re2 = np.array([j/re1 for j in vec])
        else:
            re2 = np.array([0 for j in vec])
        return VII(re2, re1)
    
    # Convert scalar to VII format data
    @staticmethod
    def load_intnsity(sca):
        re1 = np.array([0, 0, 0])
        re2 = sca
        return VII(re1, re2)
    
    # Convert VII format data into 3D vectors
    def back_vector(self):
        re1 = self.direction
        re2 = self.mould
        re = np.array([re2*i for i in re1])
        return re
    
    # Define "+" operation
    def __add__(self, A):
        if self.type=='vector' and A.type=='vector':
            re1 = VII.back_vector(self)
            re2 = VII.back_vector(A)
            re3 = re1 + re2
            re = VII.load_vector(re3)
        elif self.type=='intnsity' and A.type=='intnsity':
            re1 = np.array([0, 0, 0])
            re2 = self.mould + A.mould
            re = VII(re1, re2)
        else:
            re = 'Error'
        return re
    
    # Define "-" operation
    def __sub__(self, A):
        if self.type=='vector' and A.type=='vector':
            re1 = VII.back_vector(self)
            re2 = VII.back_vector(A)
            re3 = re1 - re2
            re = VII.load_vector(re3)
        elif self.type=='intnsity' and A.type=='intnsity':
            re1 = np.array([0, 0, 0])
            re2 = self.mould - A.mould
            re = VII(re1, re2)
        else:
            re = 'Error'
        return re
    
    # Define "*" operation
    def __mul__(self, A):
        if self.type=='intnsity' and A.type=='intnsity':
            re1 = np.array([0, 0, 0])
            re2 = self.mould * A.mould
            re = VII(re1, re2)
        elif self.type=='vector' and A.type=='intnsity':
            re1 = A.mould
            re2 = VII.back_vector(self)
            re3 = np.array([re1*i for i in re2])
            re = VII.load_vector(re3)
        elif self.type=='intnsity' and A.type=='vector':
            re1 = self.mould
            re2 = VII.back_vector(A)
            re3 = np.array([re1*i for i in re2])
            re = VII.load_vector(re3)
        elif self.type=='nabla' and A.type=='intnsity':
            re1 = A.mould
            re2 = np.array([diff(re1, x), diff(re1, y), diff(re1, z)])
            re = VII.load_vector(re2)
        else:
            re = 'Error'
        return re
    
    # Define "/" operation
    def __truediv__(self, A):
        if self.type=='intnsity' and A.type=='intnsity':
            re1 = np.array([0, 0, 0])
            re2 = self.mould / A.mould
            re = VII(re1, re2)
        elif self.type=='vector' and A.type=='intnsity':
            re1 = A.mould
            re2 = VII.back_vector(self)
            re3 = np.array([i/re1 for i in re2])
            re = VII.load_vector(re3)
        else:
            re = 'Error'
        return re
    
    # Define Dot product operation
    def dot(self, A):
        if self.type=='vector' and A.type=='vector':
            re1 = np.dot(self.direction, A.direction)
            re2 = self.mould * A.mould
            re = re1 * re2
        elif self.type=='nabla' and A.type=='vector':
            re1 = VII.back_vector(A)
            re2 = diff(re1[0], x) + diff(re1[1], y) + diff(re1[2], z)
            re = VII.load_intnsity(re2)
        return re
    
    # Define Cross product operation
    def cross(self, A):
        if self.type=='vector' and A.type=='vector':
            re1 = np.cross(self.direction, A.direction)
            re2 = self.mould * A.mould
            re3 = VII(re1, re2)
            re = VII.load_vector(VII.back_vector(re3))
        elif self.type=='nabla' and A.type=='vector':
            re1 = VII.back_vector(A)
            re2 = np.array([diff(re1[2], y)-diff(re1[1], z), diff(re1[0], z)-diff(re1[2], x), diff(re1[1], x)-diff(re1[0], y)])
            re = VII.load_vector(re2)
        return re
    
    # show VII type
    def __repr__(self):
        return str((self.direction, self.mould))

In [2]:
x, y, z = symbols('x, y, z')

L = x**2 + y**3 - z/2
M = [x, y, z]
N = [x**2, y**2, z**2]
A = [3, 4, 0]
B = [4, 3, 0]
a = VII.load_vector(A)
b = VII.load_vector(B)
m = VII.load_vector(M)
n = VII.load_vector(N)
l = VII.load_intnsity(L)
p = VII.load_intnsity(x)
q = VII.load_intnsity(y)
nab = VII.nabla()

In [3]:
nab.dot(m)

(array([0, 0, 0]), 3)

In [4]:
nab.dot(n)

(array([0, 0, 0]), 2*x + 2*y + 2*z)

In [5]:
nab.cross(m)

(array([0, 0, 0]), 0)

In [6]:
nab.cross(p*m).back_vector()

array([0, -z, y], dtype=object)

In [7]:
nab.cross(n/q).back_vector()

array([-z**2/y**2, 0, x**2/y**2], dtype=object)

In [8]:
(nab * l).back_vector()

array([2*x, 3*y**2, -1/2], dtype=object)

In [9]:
nana = VII(np.array([x, y, z]), 0)
nana.type

'nabla'